In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import rasterio
import numpy as np
import torch.nn as nn
import torch.optim as optim
from skimage.transform import resize
from tqdm import tqdm
import pandas as pd
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import mapping
from rasterio.features import geometry_mask
import torch.nn.functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tif_path = "F:/12_PET/2001/PET_2001_01_01.tif"
with rasterio.open(tif_path) as src:
        image_data = src.read(1)  # 读取第一个波段
        transform = src.transform
        crs = src.crs
        bounds = src.bounds
        height, width = src.height, src.width
        profile = src.profile  # 获取文件元数据以便后续导出
    
shp_path = "C:/Users/Administrator/Desktop/边界/中国/china.shp"
shapefile = gpd.read_file(shp_path)
shapefile = shapefile.to_crs(crs)
geometries = [mapping(geometry) for geometry in shapefile.geometry]
mask = geometry_mask(geometries, transform=transform, invert=True, out_shape=(height, width))

Using device: cuda


In [4]:
class SuperResolutionDataset(Dataset):
    def __init__(self, tem_dirs, tem_min_dirs, tem_max_dirs, pr_dirs, wind_dirs, ps_dirs, solar_dirs, ther_dirs, rh_dirs, eva_dirs, transform=None):
        self.tem_files = self._get_all_files(tem_dirs)
        self.tem_min_files = self._get_all_files(tem_min_dirs)
        self.tem_max_files = self._get_all_files(tem_max_dirs)
        self.pr_files = self._get_all_files(pr_dirs)
        self.wind_files = self._get_all_files(wind_dirs)
        self.ps_files = self._get_all_files(ps_dirs)
        self.solar_files = self._get_all_files(solar_dirs)
        self.ther_files = self._get_all_files(ther_dirs)
        self.rh_files = self._get_all_files(rh_dirs)
        self.eva_files = self._get_all_files(eva_dirs)
        
        self.transform = transform

    def _get_all_files(self, dirs):
        all_files = []
        for directory in dirs:
            for filename in sorted(os.listdir(directory)):  # 排序保证匹配
                file_path = os.path.join(directory, filename)
                if os.path.isfile(file_path):  # 确保是文件
                    all_files.append(file_path)
        return all_files

    def __len__(self):
        return len(self.tem_files)

    def __getitem__(self, idx):
        tem_path = self.tem_files[idx]
        tem_min_path = self.tem_min_files[idx]
        tem_max_path = self.tem_max_files[idx]
        pr_path = self.pr_files[idx]
        wind_path = self.wind_files[idx]
        ps_path = self.ps_files[idx]
        solar_path = self.solar_files[idx]
        ther_path = self.ther_files[idx]
        rh_path = self.rh_files[idx]
        eva_path = self.eva_files[idx]
        
        with rasterio.open(tem_path) as tem_dataset:
            tem_image = tem_dataset.read(1)  
        with rasterio.open(tem_min_path) as tem_min_dataset:
            tem_min_image = tem_min_dataset.read(1)  
        with rasterio.open(tem_max_path) as tem_max_dataset:
            tem_max_image = tem_max_dataset.read(1)  
        with rasterio.open(pr_path) as pr_dataset:
            pr_image = pr_dataset.read(1)
            pr_image = pr_image*1000
        with rasterio.open(wind_path) as wind_dataset:
            wind_image = wind_dataset.read(1)  
        with rasterio.open(ps_path) as ps_dataset:
            ps_image = ps_dataset.read(1)  
            ps_image = ps_image/1000
        with rasterio.open(solar_path) as solar_dataset:
            solar_image = solar_dataset.read(1) 
            solar_image = solar_image/86400
        with rasterio.open(ther_path) as ther_dataset:
            ther_image = ther_dataset.read(1) 
            ther_image = ther_image/86400
        with rasterio.open(rh_path) as rh_dataset:
            rh_image = rh_dataset.read(1) 
        with rasterio.open(eva_path) as eva_dataset:
            eva_image = eva_dataset.read(1) 

        # 如果有 transform，应用转换
        if self.transform:
            tem_image = self.transform(tem_image)
            tem_min_image = self.transform(tem_min_image)
            tem_max_image = self.transform(tem_max_image)
            pr_image = self.transform(pr_image)
            wind_image = self.transform(wind_image)
            ps_image = self.transform(ps_image)
            solar_image = self.transform(solar_image)
            ther_image = self.transform(ther_image)
            rh_image = self.transform(rh_image)
            eva_image = self.transform(eva_image)

        else:
        # 如果没有 transform，转换为 PyTorch 张量
            tem_image = torch.from_numpy(tem_image).float()  # (1, H, W)
            tem_min_image = torch.from_numpy(tem_min_image).float()  # (1, H, W)
            tem_max_image = torch.from_numpy(tem_max_image).float()  # (1, H, W)
            pr_image = torch.from_numpy(pr_image).float()  # (1, H, W)
            wind_image = torch.from_numpy(wind_image).float()  # (1, H, W)
            ps_image = torch.from_numpy(ps_image).float()  # (1, H, W)
            solar_image = torch.from_numpy(solar_image).float()  # (1, H, W)
            ther_image = torch.from_numpy(ther_image).float()  # (1, H, W)
            rh_image = torch.from_numpy(rh_image).float()  # (1, H, W)
            eva_image = torch.from_numpy(eva_image).float()  # (1, H, W)

        lr_image =  torch.cat([tem_image, tem_min_image,tem_max_image,pr_image,wind_image,ps_image,solar_image,ther_image,rh_image], dim=0)
            
        return lr_image, eva_image
# 定义数据增强和预处理操作
data_transforms = transforms.Compose([
    transforms.ToTensor(),  # 将 NumPy 数组转换为 PyTorch 张量
])

def load_tiff_with_rasterio(file_path):
    with rasterio.open(file_path) as src:
        img = src.read(1)  # 读取第一个波段
        img = img
    return img

In [5]:
year = list(range(2000,2014))
tem_directories = []
tem_min_directories = []
tem_max_directories = []
pr_directories = []
wind_directories = []
ps_directories = []
solar_directories = []
ther_directories = []
rh_directories = []
eva_directories = []
for a_index,a in enumerate(year):
    tem_path = f'F:/6_climate_100_59_33/1_tem/{a}'
    tem_directories.append(tem_path)
    tem_min_path = f'F:/6_climate_100_59_33/3_tem_min/{a}'
    tem_min_directories.append(tem_min_path)
    tem_max_path = f'F:/6_climate_100_59_33/2_tem_max/{a}'
    tem_max_directories.append(tem_max_path)
    pr_path = f'F:/6_climate_100_59_33/7_tol_per/{a}'
    pr_directories.append(pr_path)
    wind_path = f'F:/6_climate_100_59_33/4_wind_u/{a}'
    wind_directories.append(wind_path)
    ps_path = f'F:/6_climate_100_59_33/6_ps/{a}'
    ps_directories.append(ps_path)
    solar_path = f'F:/6_climate_100_59_33/8_solar_rad/{a}'
    solar_directories.append(solar_path)
    ther_path = f'F:/6_climate_100_59_33/9_ther_rad/{a}'
    ther_directories.append(ther_path)
    rh_path = f'F:/6_climate_100_59_33/11_rh/{a}'
    rh_directories.append(rh_path)
    eva_path = f'F:/12_PET/{a}'
    eva_directories.append(eva_path)
dataset_train = SuperResolutionDataset(tem_dirs = tem_directories, tem_min_dirs = tem_min_directories, tem_max_dirs = tem_max_directories, 
                                       pr_dirs = pr_directories, wind_dirs = wind_directories, ps_dirs = ps_directories, solar_dirs = solar_directories,
                                       ther_dirs = ther_directories, rh_dirs = rh_directories, eva_dirs = eva_directories, transform=data_transforms)
dataloader_train = DataLoader(dataset_train, batch_size=8, shuffle=True, num_workers=0)


year = list(range(2014,2019))
tem_directories = []
tem_min_directories = []
tem_max_directories = []
pr_directories = []
wind_directories = []
ps_directories = []
solar_directories = []
ther_directories = []
rh_directories = []
eva_directories = []
for a_index,a in enumerate(year):
    tem_path = f'F:/6_climate_100_59_33/1_tem/{a}'
    tem_directories.append(tem_path)
    tem_min_path = f'F:/6_climate_100_59_33/3_tem_min/{a}'
    tem_min_directories.append(tem_min_path)
    tem_max_path = f'F:/6_climate_100_59_33/2_tem_max/{a}'
    tem_max_directories.append(tem_max_path)
    pr_path = f'F:/6_climate_100_59_33/7_tol_per/{a}'
    pr_directories.append(pr_path)
    wind_path = f'F:/6_climate_100_59_33/4_wind_u/{a}'
    wind_directories.append(wind_path)
    ps_path = f'F:/6_climate_100_59_33/6_ps/{a}'
    ps_directories.append(ps_path)
    solar_path = f'F:/6_climate_100_59_33/8_solar_rad/{a}'
    solar_directories.append(solar_path)
    ther_path = f'F:/6_climate_100_59_33/9_ther_rad/{a}'
    ther_directories.append(ther_path)
    rh_path = f'F:/6_climate_100_59_33/11_rh/{a}'
    rh_directories.append(rh_path)
    eva_path = f'F:/12_PET/{a}'
    eva_directories.append(eva_path)
dataset_val = SuperResolutionDataset(tem_dirs = tem_directories, tem_min_dirs = tem_min_directories, tem_max_dirs = tem_max_directories, 
                                       pr_dirs = pr_directories, wind_dirs = wind_directories, ps_dirs = ps_directories, solar_dirs = solar_directories,
                                       ther_dirs = ther_directories, rh_dirs = rh_directories, eva_dirs = eva_directories, transform=data_transforms)
dataloader_val = DataLoader(dataset_val, batch_size=8, shuffle=True, num_workers=0)


year = list(range(2019,2024))
tem_directories = []
tem_min_directories = []
tem_max_directories = []
pr_directories = []
wind_directories = []
ps_directories = []
solar_directories = []
ther_directories = []
rh_directories = []
eva_directories = []
for a_index,a in enumerate(year):
    tem_path = f'F:/6_climate_100_59_33/1_tem/{a}'
    tem_directories.append(tem_path)
    tem_min_path = f'F:/6_climate_100_59_33/3_tem_min/{a}'
    tem_min_directories.append(tem_min_path)
    tem_max_path = f'F:/6_climate_100_59_33/2_tem_max/{a}'
    tem_max_directories.append(tem_max_path)
    pr_path = f'F:/6_climate_100_59_33/7_tol_per/{a}'
    pr_directories.append(pr_path)
    wind_path = f'F:/6_climate_100_59_33/4_wind_u/{a}'
    wind_directories.append(wind_path)
    ps_path = f'F:/6_climate_100_59_33/6_ps/{a}'
    ps_directories.append(ps_path)
    solar_path = f'F:/6_climate_100_59_33/8_solar_rad/{a}'
    solar_directories.append(solar_path)
    ther_path = f'F:/6_climate_100_59_33/9_ther_rad/{a}'
    ther_directories.append(ther_path)
    rh_path = f'F:/6_climate_100_59_33/11_rh/{a}'
    rh_directories.append(rh_path)
    eva_path = f'F:/12_PET/{a}'
    eva_directories.append(eva_path)
dataset_test = SuperResolutionDataset(tem_dirs = tem_directories, tem_min_dirs = tem_min_directories, tem_max_dirs = tem_max_directories, 
                                       pr_dirs = pr_directories, wind_dirs = wind_directories, ps_dirs = ps_directories, solar_dirs = solar_directories,
                                       ther_dirs = ther_directories, rh_dirs = rh_directories, eva_dirs = eva_directories, transform=data_transforms)
dataloader_test = DataLoader(dataset_test, batch_size=8, shuffle=True, num_workers=0)

In [6]:
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in tqdm(dataloader):
        inputs  = inputs.float().to(device)
        targets = targets.float().to(device)

        inputs[torch.isnan(inputs)] = 0
        targets[torch.isnan(targets)] = 0

        assert not torch.isnan(inputs).any(), "Input contains NaN values"
        assert not torch.isnan(targets).any(), "Target contains NaN values"
        
        optimizer.zero_grad()
        outputs_up = model(inputs)

        loss = criterion(outputs_up, targets).to(device)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def validate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader):
            inputs = inputs.float().to(device)
            targets =  targets.float().to(device)
            inputs[torch.isnan(inputs)] = 0
            targets[torch.isnan(targets)] = 0

            outputs = model(inputs)

            loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss


In [7]:
import torch
import torch.nn as nn

class ChannelAttention(nn.Module):
    def __init__(self, num_features, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Sequential(
            nn.Conv2d(num_features, num_features // reduction, 1, padding=0, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_features // reduction, num_features, 1, padding=0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.conv(y)
        return x * y

class RCAB(nn.Module):
    def __init__(self, num_features, reduction):
        super(RCAB, self).__init__()
        self.body = nn.Sequential(
            nn.Conv2d(num_features, num_features, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_features, num_features, 3, padding=1),
            ChannelAttention(num_features, reduction)
        )

    def forward(self, x):
        return x + self.body(x)

class ResidualGroup(nn.Module):
    def __init__(self, num_features, num_blocks, reduction):
        super(ResidualGroup, self).__init__()
        self.body = nn.Sequential(
            *[RCAB(num_features, reduction) for _ in range(num_blocks)],
            nn.Conv2d(num_features, num_features, 3, padding=1)
        )

    def forward(self, x):
        return x + self.body(x)

class RCAN(nn.Module):
    def __init__(self, num_features=64, num_groups=10, num_blocks=20, reduction=16, scale=10, in_channels=9, out_channels=1):
        super(RCAN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, num_features, 3, padding=1)
        
        self.body = nn.Sequential(
            *[ResidualGroup(num_features, num_blocks, reduction) for _ in range(num_groups)],
            nn.Conv2d(num_features, num_features, 3, padding=1)
        )
        
        self.upsample = nn.Sequential(
            nn.Conv2d(num_features, num_features * (scale ** 2), 3, padding=1),
            nn.PixelShuffle(scale)
        )
        
        self.conv2 = nn.Conv2d(num_features, out_channels, 3, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        res = self.body(x)
        res += x
        x = self.upsample(res)
        x = self.conv2(x)
        return x

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = RCAN(num_features=64, num_groups=10, num_blocks=20, reduction=16, scale=10, in_channels=9, out_channels=1).to(device)

learning_rate = 0.0001
criterion = nn.L1Loss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)

num_epochs = 100
best_loss = float('inf')

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Model is training on: {next(model.parameters()).device}')
    train_loss = train_model(model, dataloader_train, criterion, optimizer, device)
    val_loss = validate_model(model, dataloader_val, criterion, device)
    print(f'Train Loss: {train_loss:.4f} | Validation Loss: {val_loss:.4f}')
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'G:/16_ECMWF/13_DL_modle/4_climate_590_330/RCAN/RCAN_pet.pth')
        print('Model saved!')


cuda
Epoch 1/100
Model is training on: cuda:0


100%|██████████| 229/229 [01:26<00:00,  2.64it/s]


Train Loss: 7.1656 | Validation Loss: 1.6125
Model saved!
Epoch 2/100
Model is training on: cuda:0


100%|██████████| 229/229 [01:21<00:00,  2.81it/s]


Train Loss: 1.1342 | Validation Loss: 0.8772
Model saved!
Epoch 3/100
Model is training on: cuda:0


...

Train Loss: 0.1420 | Validation Loss: 0.1504
Epoch 99/100
Model is training on: cuda:0


100%|██████████| 229/229 [01:15<00:00,  3.05it/s]


Train Loss: 0.1418 | Validation Loss: 0.1578
Epoch 100/100
Model is training on: cuda:0


100%|██████████| 229/229 [01:15<00:00,  3.05it/s]

Train Loss: 0.1416 | Validation Loss: 0.1565
